In [26]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Ing2Vec_ingredient2vec_results_in_txt_ing2vec_word2vec_cbow_1mil_300_8.txt to Ing2Vec_ingredient2vec_results_in_txt_ing2vec_word2vec_cbow_1mil_300_8.txt
User uploaded file "Ing2Vec_ingredient2vec_results_in_txt_ing2vec_word2vec_cbow_1mil_300_8.txt" with length 569690 bytes


In [7]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
from __future__ import print_function

# Handle data
import json
import operator
import collections
import re

#!pip3 install -U gensim
#!pip install pandas
import numpy as np
import pandas as pd

from pprint import pprint

# Model Algorithms
from gensim.models.word2vec import Word2Vec

# Modelling Helpers, see above the description
from sklearn.manifold import TSNE


from gensim.models import KeyedVectors
# PCA for dimensionality reduction
from sklearn.decomposition import PCA

# Visualisation
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import os

In [18]:
os.chdir("/content")
!ls
df3 = pd.read_csv('raw_data_dbpedia_ingredients.txt')
list(df3.columns)
df3.head(100)

Ing2Vec_models_word2vec_models_cbow_ing2vec_word2vec_cbow_300_8.model
raw_data_dbpedia_ingredients.txt
sample_data


,Alkanna tinctoria
0,Apricot
1,Asparagus
2,Bamboo
3,Banana
4,Beef
5,Beeswax
6,Berry
7,Brandy
8,Bread
9,Breadfruit


# New Section

In [21]:
os.chdir("/content")
!ls
loaded_model = Word2Vec.load("Ing2Vec_models_word2vec_models_cbow_ing2vec_word2vec_cbow_300_8.model")
print(loaded_model)

Ing2Vec_models_word2vec_models_cbow_ing2vec_word2vec_cbow_300_8.model
raw_data_dbpedia_ingredients.txt
sample_data
Word2Vec(vocab=1210, size=300, alpha=0.025)


In [28]:
loaded_model.most_similar(u'pine nut')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('prosciutto', 0.5174710154533386),
 ('farro', 0.5077522397041321),
 ('pecorino', 0.4704417288303375),
 ('fava bean', 0.46095719933509827),
 ('ricotta', 0.4499695897102356),
 ('durum wheat', 0.44967925548553467),
 ('edible flowers', 0.44870835542678833),
 ('escarole', 0.43923795223236084),
 ('leafy greens', 0.4373020529747009),
 ('broad bean', 0.4324149489402771)]

In [29]:
#find_substitutes_of_ingredients = [u'heavy cream', u'butter', u'sugar', u'brown sugar', u'cream cheese', u'rice', u'bread', u'vegetable oil', u'coconut oil']
find_substitutes_of_ingredients = []
os.chdir("/content")
!ls
with open("raw_data_dbpedia_ingredients.txt") as dbpedia_ingredients:
    for ingredient in dbpedia_ingredients:
        ingredient = ingredient.replace("\"", "")
        ingredient = ingredient.replace("\\", "")
        ingredient = ingredient.replace("\n", "")
        find_substitutes_of_ingredients.append(ingredient.lower())

Ing2Vec_ingredient2vec_results_in_txt_ing2vec_word2vec_cbow_1mil_300_8.txt
Ing2Vec_models_word2vec_models_cbow_ing2vec_word2vec_cbow_300_8.model
raw_data_dbpedia_ingredients.txt
sample_data


In [30]:
os.chdir("/content")
!ls
logFile = open('Ing2Vec_ingredient2vec_results_in_txt_ing2vec_word2vec_cbow_1mil_300_8.txt', 'w')
for ingredient in find_substitutes_of_ingredients:
    ingredient_str = f"\n\nMost similar ingredients to %s :\n\n'"%(ingredient)
    logFile.write(ingredient_str)
    #pprint(loaded_model.most_similar(ingredient))
    #print('\n')
    try:
        pprint(loaded_model.most_similar(ingredient), logFile)
    except:
        error_msg = f"all ngrams for word %s absent from model"%(ingredient)
        logFile.write(error_msg)

Ing2Vec_ingredient2vec_results_in_txt_ing2vec_word2vec_cbow_1mil_300_8.txt
Ing2Vec_models_word2vec_models_cbow_ing2vec_word2vec_cbow_300_8.model
raw_data_dbpedia_ingredients.txt
sample_data


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
